In [2]:
import torch

print (f"using torch {torch.__version__}")
if torch.cuda.is_available():
    device="cuda"

elif torch.backends.mps.is_available():
    device="mps"

else:
    device ="cpu"

print(f"using device ={device}")


using torch 2.10.0+cu130
using device =cuda


In [3]:
train_dir = './train'
test_dir = './val'

In [4]:
from torch import nn 
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

train_transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])
test_transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

In [5]:
# load datasets
train_data = datasets.ImageFolder(root=train_dir, transform=train_transform)
test_data = datasets.ImageFolder(root=test_dir,transform=test_transform)

In [6]:
# create data loader
batch_size = 16
train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True)
test_loader = DataLoader(test_data,batch_size=batch_size,shuffle=False)

In [7]:
print(f"Training data contains {len(train_data) } images")

Training data contains 9469 images


In [8]:
class_names_idx=train_data.class_to_idx
print(class_names_idx)

{'n01440764': 0, 'n02102040': 1, 'n02979186': 2, 'n03000684': 3, 'n03028079': 4, 'n03394916': 5, 'n03417042': 6, 'n03425413': 7, 'n03445777': 8, 'n03888257': 9}


In [9]:
from model import CustomCNN

In [10]:
# initialize model
num_classes = len(train_data.classes)
model = CustomCNN(num_classes).to(device)

In [11]:
from torchinfo import summary

model.to(device)
summary(model=model,
        input_size=(1, 3, 128, 128),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
CustomCNN (CustomCNN)                    [1, 3, 128, 128]     [1, 10]              --                   True
├─Conv2d (conv1)                         [1, 3, 128, 128]     [1, 32, 128, 128]    896                  True
├─ReLU (relu)                            [1, 32, 128, 128]    [1, 32, 128, 128]    --                   --
├─MaxPool2d (pool)                       [1, 32, 128, 128]    [1, 32, 64, 64]      --                   --
├─Conv2d (conv2)                         [1, 32, 64, 64]      [1, 64, 64, 64]      18,496               True
├─ReLU (relu)                            [1, 64, 64, 64]      [1, 64, 64, 64]      --                   --
├─MaxPool2d (pool)                       [1, 64, 64, 64]      [1, 64, 32, 32]      --                   --
├─Conv2d (conv3)                         [1, 64, 32, 32]      [1, 128, 32, 32]     73,856               True
├─ReLU (relu)         

In [12]:
#define
criterion =nn.CrossEntropyLoss()
optimizer =torch.optim.Adam(model.parameters(),lr=0.001,weight_decay=1e-4)

In [13]:
from timeit import default_timer as timer
from trainNN import train
torch.manual_seed(42)
torch.cuda.manual_seed(42)
start_time =timer()
results =train(
    model=model,
    train_dataloader =train_loader,
    test_dataloader=test_loader,
    optimizer=optimizer,
    loss_fn=criterion,
    epochs=20,
    device=device
)

end_time=timer()
print(f'[info] total training time:{end_time -start_time}')


Epochs:   5%|▌         | 1/20 [00:54<17:23, 54.90s/it]

Epoch: 1 | train_loss: 1.6231 | train_acc: 0.4498 | test_loss: 1.3469 | test_acc: 0.5544
[INFO] New best model saved to best_model.pth with test_acc: 0.5544


Epochs:  10%|█         | 2/20 [01:47<16:00, 53.34s/it]

Epoch: 2 | train_loss: 1.1156 | train_acc: 0.6359 | test_loss: 1.1235 | test_acc: 0.6396
[INFO] New best model saved to best_model.pth with test_acc: 0.6396


Epochs:  15%|█▌        | 3/20 [02:39<14:58, 52.83s/it]

...

Epoch: 18 | train_loss: 0.0875 | train_acc: 0.9743 | test_loss: 2.6106 | test_acc: 0.6398


Epochs:  95%|█████████▌| 19/20 [16:17<00:50, 50.63s/it]

Epoch: 19 | train_loss: 0.0655 | train_acc: 0.9816 | test_loss: 2.8969 | test_acc: 0.6053


Epochs: 100%|██████████| 20/20 [17:09<00:00, 51.50s/it]

Epoch: 20 | train_loss: 0.0578 | train_acc: 0.9830 | test_loss: 3.0158 | test_acc: 0.6175
[info] total training time:1029.9364702999992
